In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings

# 1. Data Preparation

In [3]:
## input FNA set ## 88701 x 20 df
fnaDEFSDf = pd.read_csv(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_NonInFNA\df_noningestedFNA24_std.csv")
#fnaDEFSDf[fnaDEFSDf.type .== 1, :]
## calculate weight ##  0: 44540, 1: 40783
Yy_FNA = fnaDEFSDf["type"]  # 0.9578; 1.0461
sampleFNAW = []
for w in Yy_FNA:
    if w == 0:
        sampleFNAW.append(0.9578)
    elif w == 1:
        sampleFNAW.append(1.0461)


In [4]:
fnaDEFSDf

,pixel_id,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,609.5114,682.5922,846.5858,633.5108,445.3172,683.5948,269.0878,215.0328,283.255,671.4661,610.5128,175.0246,type
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,0.064937,-0.160987,0.157940,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,0.631340,0.268503,-0.317168,-0.040587,0.0,0.338317,0
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,-0.265543,0.071189,0.458388,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,0.702486,0.260096,-0.317168,-0.064218,0.0,0.409220,0
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,-0.265543,0.103618,0.411193,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,0.783693,0.265866,-0.317168,-0.026163,0.0,0.434277,0
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,-0.265543,0.665530,-0.103978,0.0,0.0,-0.164278,-0.238513,0.631412,0.0,0.0,-0.123328,0.734532,0.464433,-0.317168,-0.399857,0.0,0.506886,0
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,-0.265543,0.605478,-0.103978,0.0,0.0,-0.164278,-0.238513,0.635861,0.0,0.0,-0.123328,0.714294,0.541287,-0.317168,-0.399857,0.0,0.589304,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85318,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,0.0,-0.265543,0.084344,0.215631,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,-0.086454,0.000254,0.132632,-0.399857,0.0,-0.099576,1
85319,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,0.0,-0.265543,0.470346,0.211699,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,-0.086454,-0.207168,0.277684,-0.399857,0.0,-0.099576,1
85320,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,0.0,-0.265543,0.420349,-0.103978,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,-0.086454,-0.026754,-0.317168,-0.399857,0.0,-0.099576,1
85321,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,0.0,-0.265543,0.499509,-0.103978,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,-0.086454,-0.207168,0.281786,-0.399857,0.0,-0.099576,1


In [5]:
## define functions for performace evaluation ##
    # Maximum absolute error
# mean square error (MSE) calculation
# Root mean square error (RMSE) calculation
def errorDetermination(arrRi, predictedRi):
    sumAE = 0
    maxAE = 0
    for i in range(len(predictedRi)):
        AE = abs(arrRi[i] - predictedRi[i])
        if (AE > maxAE):
            maxAE = AE
        sumAE += (AE ** 2)
        
    MSE = sumAE / len(predictedRi)
    RMSE = MSE ** 0.5
    return maxAE, MSE, RMSE

#
# Average score
def avgScore(arrAcc, cv):
    sumAcc = 0
    for acc in arrAcc:
        sumAcc += acc
    return sumAcc / cv


In [8]:
fnaDEFSDf[fnaDEFSDf.columns[1:-1]]

,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,609.5114,682.5922,846.5858,633.5108,445.3172,683.5948,269.0878,215.0328,283.255,671.4661,610.5128,175.0246
0,0.0,0.064937,-0.160987,0.157940,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,0.631340,0.268503,-0.317168,-0.040587,0.0,0.338317
1,0.0,-0.265543,0.071189,0.458388,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,0.702486,0.260096,-0.317168,-0.064218,0.0,0.409220
2,0.0,-0.265543,0.103618,0.411193,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,0.783693,0.265866,-0.317168,-0.026163,0.0,0.434277
3,0.0,-0.265543,0.665530,-0.103978,0.0,0.0,-0.164278,-0.238513,0.631412,0.0,0.0,-0.123328,0.734532,0.464433,-0.317168,-0.399857,0.0,0.506886
4,0.0,-0.265543,0.605478,-0.103978,0.0,0.0,-0.164278,-0.238513,0.635861,0.0,0.0,-0.123328,0.714294,0.541287,-0.317168,-0.399857,0.0,0.589304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85318,0.0,-0.265543,0.084344,0.215631,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,-0.086454,0.000254,0.132632,-0.399857,0.0,-0.099576
85319,0.0,-0.265543,0.470346,0.211699,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,-0.086454,-0.207168,0.277684,-0.399857,0.0,-0.099576
85320,0.0,-0.265543,0.420349,-0.103978,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,-0.086454,-0.026754,-0.317168,-0.399857,0.0,-0.099576
85321,0.0,-0.265543,0.499509,-0.103978,0.0,0.0,-0.164278,-0.238513,-0.067201,0.0,0.0,-0.123328,-0.086454,-0.207168,0.281786,-0.399857,0.0,-0.099576


In [9]:
import warnings
warnings.filterwarnings('ignore')

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [10]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [11]:
df_ROI_for_ML_Opti_FNA = fnaDEFSDf.set_index('pixel_id')

## 2.2. PCA Analysis

In [14]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_FNA):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_FNA.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1', size=20)
    plt.ylabel('PC2', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_NonInFNA/PCA_noningestedFNA.tif", bbox_inches = 'tight')

    """components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()"""

In [15]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [16]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [17]:
y_FNA = df_ROI_for_ML_Opti_FNA["type"]

## 3.2 Prediction by Model 1+2 (Train 10:10, Val 4:4, Test 3:3)

In [18]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [19]:
import joblib as jl

## 4.1 Prediction

In [23]:
## deploy models for FNA set ##
model_xgb = jl.load(r"H:\3_output_raMSIn\\modelXGBoost.joblib")
#
## deploy model ##
predicted0n1_FNA = model_xgb.predict(X_FNA)
fnaDEFSDf["predicted0n1"] = predicted0n1_FNA
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_NonInFNA\df_noningestedFNA24_std_0n1.csv"
fnaDEFSDf.to_csv(savePath)

## 4.2 p(0) p(1)

In [24]:
pTP_FNA = model_xgb.predict_proba(X_FNA)
fnaDEFSDf["p(0)"] = list(pd.DataFrame(pTP_FNA)[0])
fnaDEFSDf["p(1)"] = list(pd.DataFrame(pTP_FNA)[1])
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_NonInFNA\df_noningestedFNA24_std_0n1_pTP.csv"
fnaDEFSDf.to_csv(savePath)